# ETL

En esta jupyter notebook se desarrolla la extracción, transformación y carga de los conjuntos de datos.

## Importaciones

In [2]:
import pandas as pd
import json
import gzip
import numpy as np
import ast
from tqdm import tqdm
import os
from fastapi import FastAPI

import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_distances
from dateutil import parser

"""import utils"""
import warnings
warnings.filterwarnings("ignore")

## ETL de `output_steam_games`

### 1.1 Extracción de los datos

In [172]:
# Importar el Data set
dataset = pd.read_json('output_steam_games.json', lines=True)


In [173]:
#Mostramos el Dataset
dataset

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


### 1.2 Transformación de los datos


Analizamos la data si hay ID repetidas


In [174]:
call_id_counts = dataset['id'].value_counts()
call_id_counts

612880.0    2
761140.0    1
530200.0    1
518690.0    1
513460.0    1
           ..
676060.0    1
494160.0    1
215280.0    1
667090.0    1
681550.0    1
Name: id, Length: 32132, dtype: int64

Verificamos cuantos valores se repiten

In [175]:
id_mas_de_una_vez = call_id_counts[call_id_counts > 1].index.tolist()

print(id_mas_de_una_vez)


[612880.0]


Procedemos a encontrar la fila que se repite para luego analizarla

In [176]:
fila_resultado = dataset[dataset['id'] == 612880.0]
fila_resultado

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games
102883,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/Wolfe...,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


Observamos que ambas filas se repiten, pasamos a eliminar una de ellas

In [177]:
dataset = dataset.drop(102883)

Verificamos nuevamente por el ID REPETIDO, Observamos que ya se encuentra solo 1 fila.

In [178]:
fila_resultado = dataset[dataset['id'] == 612880.0]
fila_resultado

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
102204,Bethesda Softworks,[Action],Wolfenstein II: The New Colossus,Wolfenstein II: The New Colossus,http://store.steampowered.com/app/612880/,2017-10-26,"[Action, FPS, Gore, Violent, Alternate History...",http://steamcommunity.com/app/612880/reviews/?...,"[Single-player, Steam Achievements, Full contr...",59.99,0.0,612880.0,Machine Games


Ahora procedo a eliminar los NAN de las primeras filas(obtuve el resultado viendo hasta donde llegaban estas filas vacias en el CSV)

In [179]:
dataset = dataset.drop(range(0, 88384))
dataset

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88384,None,None,None,None,http://store.steampowered.com/,None,None,None,None,19.99,0.0,NaN,None
88385,2K Games,[Strategy],Railroad Tycoon II Platinum,Railroad Tycoon II Platinum,http://store.steampowered.com/app/7620/Railroa...,1998-10-31,"[Strategy, Management, Trains, Simulation, Sin...",http://steamcommunity.com/app/7620/reviews/?br...,[Single-player],4.99,0.0,7620.0,PopTop
88386,None,"[Casual, Indie]",RIP - Trilogy™,RIP - Trilogy™,http://store.steampowered.com/app/2540/RIP__Tr...,2007-06-01,"[Indie, Casual, Arcade, Shooter, Action, 2D, S...",http://steamcommunity.com/app/2540/reviews/?br...,"[Single-player, Multi-player, Co-op]",4.99,0.0,2540.0,Elephant Games
88387,2K Games,[Strategy],X-COM: Terror From the Deep,X-COM: Terror From the Deep,http://store.steampowered.com/app/7650/XCOM_Te...,1995-04-01,"[Strategy, Classic, Sci-fi, Turn-Based, Turn-B...",http://steamcommunity.com/app/7650/reviews/?br...,[Single-player],4.99,0.0,7650.0,"MicroProse Software, Inc"
88388,"Square Enix, Feral Interactive (Mac)","[Action, Adventure]",Tomb Raider: Anniversary,Tomb Raider: Anniversary,http://store.steampowered.com/app/8000/Tomb_Ra...,2007-06-05,"[Action, Adventure, Female Protagonist, Puzzle...",http://steamcommunity.com/app/8000/reviews/?br...,"[Single-player, Partial Controller Support]",8.99,0.0,8000.0,"Crystal Dynamics,Feral Interactive (Mac)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,Colony On Mars,http://store.steampowered.com/app/773640/Colon...,2018-01-04,"[Strategy, Indie, Casual, Simulation]",http://steamcommunity.com/app/773640/reviews/?...,"[Single-player, Steam Achievements]",1.99,0.0,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,LOGistICAL: South Africa,http://store.steampowered.com/app/733530/LOGis...,2018-01-04,"[Strategy, Indie, Casual]",http://steamcommunity.com/app/733530/reviews/?...,"[Single-player, Steam Achievements, Steam Clou...",4.99,0.0,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,0.0,610660.0,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,0.0,658870.0,"xropi,stev3ns"


Se Procede a Eliminar la colummnas que no vamos a necesitar

In [180]:
dataset.drop(['title',	'url',	'tags', 'specs', 'early_access', 'reviews_url'], axis=1, inplace=True)

Despues de un analisis de datos desde el dataset, determinamos que si eliminamos los nulos, no hay informacion relevante que se pierda haciendo un `dropna()`

In [181]:
dataset = dataset.dropna()
dataset

,publisher,genres,app_name,release_date,price,id,developer
88385,2K Games,[Strategy],Railroad Tycoon II Platinum,1998-10-31,4.99,7620.0,PopTop
88387,2K Games,[Strategy],X-COM: Terror From the Deep,1995-04-01,4.99,7650.0,"MicroProse Software, Inc"
88388,"Square Enix, Feral Interactive (Mac)","[Action, Adventure]",Tomb Raider: Anniversary,2007-06-05,8.99,8000.0,"Crystal Dynamics,Feral Interactive (Mac)"
88389,Capcom,[Action],Devil May Cry® 3 Special Edition,2006-05-23,19.99,6550.0,"CAPCOM Co., Ltd."
88390,Telltale Games,"[Action, Adventure]",Sam & Max 101: Culture Shock,2006-10-17,19.99,8200.0,Telltale Games
...,...,...,...,...,...,...,...
120439,Bidoniera Games,"[Action, Adventure, Casual, Indie]",Kebab it Up!,2018-01-04,1.99,745400.0,Bidoniera Games
120440,Ghost_RUS Games,"[Casual, Indie, Simulation, Strategy]",Colony On Mars,2018-01-04,1.99,773640.0,"Nikita ""Ghost_RUS"""
120441,Sacada,"[Casual, Indie, Strategy]",LOGistICAL: South Africa,2018-01-04,4.99,733530.0,Sacada
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,2018-01-04,1.99,610660.0,Laush Dmitriy Sergeevich


Cambiamos el nombre de la columna `id` renombrandola `item_id` para futuros merges

In [182]:
dataset = dataset.rename(columns={'id': 'item_id'})

Convierte todos los valores de tipo cadena a 0 en la columna `price`

In [183]:
columna_a_convertir = 'price'

dataset.loc[dataset[columna_a_convertir].apply(lambda x: isinstance(x, str)), columna_a_convertir] = 0

Se guarda como formato .Parquet para el merge

In [184]:
dataset.to_parquet('output_steam_games.parquet', index=False)

Se guarda como formato .CSV para el merge

In [185]:
dataset.to_csv('output_steam_games.csv', index=False)

In [186]:
dataset['genres'].value_counts()


[Action]                                                                                                        1576
[Action, Indie]                                                                                                 1394
[Simulation]                                                                                                    1268
[Strategy]                                                                                                       926
[Action, Adventure, Indie]                                                                                       925
                                                                                                                ... 
[Adventure, Casual, Indie, Racing, RPG, Simulation, Sports, Strategy]                                              1
[Animation &amp; Modeling, Audio Production, Design &amp; Illustration, Software Training, Video Production]       1
[Action, Free to Play, Simulation]                              

In [187]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22767 entries, 88385 to 120443
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     22767 non-null  object 
 1   genres        22767 non-null  object 
 2   app_name      22767 non-null  object 
 3   release_date  22767 non-null  object 
 4   price         22767 non-null  object 
 5   item_id       22767 non-null  float64
 6   developer     22767 non-null  object 
dtypes: float64(1), object(6)
memory usage: 1.4+ MB


### ETL de `australian_user_items`

### 1.1 Extracción de los datos

In [188]:
data_list = []
#Ruta del archivo JSON
file_path = 'australian_users_items.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
data_it = pd.DataFrame(data_list)


In [189]:
#Mostramos el dataframe
data_it

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


### 2.2 Transformación de los datos
#### La columna "items" alberga datos anidados en forma de una lista de diccionarios. En este conjunto de datos, se utiliza el método de normalización, donde la columna "items" sirve como columna organizadora. De esta manera, se obtienen las claves de los diccionarios como columnas individuales.

In [190]:
# Normalizamos la columna 'items' deL DATASET
data_it1 = data_it.explode(['items'])
data_it2 = pd.json_normalize(data_it1['items']).set_index(data_it1['items'].index)
data_it3= pd.concat([data_it2, data_it1], axis=1)


In [191]:
# Mostramos el Dataframe normalizado
data_it3

,item_id,item_name,playtime_forever,playtime_2weeks,user_id,items_count,steam_id,user_url,items
0,10,Counter-Strike,6.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,20,Team Fortress Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,30,Day of Defeat,7.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,40,Deathmatch Classic,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,50,Half-Life: Opposing Force,0.0,0.0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...,...,...,...,...
88308,373330,All Is Dust,0.0,0.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '373330', 'item_name': 'All Is Dus..."
88308,388490,One Way To Die: Steam Edition,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '388490', 'item_name': 'One Way To..."
88308,521570,You Have 10 Seconds 2,4.0,4.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '521570', 'item_name': 'You Have 1..."
88308,519140,Minds Eyes,3.0,3.0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


Se eliminan las columnas que no se consideran relevantes para el posterior analisis

In [192]:
data_it3.drop(['items_count','user_url','item_name','steam_id'], axis=1, inplace=True)

In [193]:
data_it3

,item_id,playtime_forever,playtime_2weeks,user_id,items
0,10,6.0,0.0,76561197970982479,"{'item_id': '10', 'item_name': 'Counter-Strike..."
0,20,0.0,0.0,76561197970982479,"{'item_id': '20', 'item_name': 'Team Fortress ..."
0,30,7.0,0.0,76561197970982479,"{'item_id': '30', 'item_name': 'Day of Defeat'..."
0,40,0.0,0.0,76561197970982479,"{'item_id': '40', 'item_name': 'Deathmatch Cla..."
0,50,0.0,0.0,76561197970982479,"{'item_id': '50', 'item_name': 'Half-Life: Opp..."
...,...,...,...,...,...
88308,373330,0.0,0.0,76561198329548331,"{'item_id': '373330', 'item_name': 'All Is Dus..."
88308,388490,3.0,3.0,76561198329548331,"{'item_id': '388490', 'item_name': 'One Way To..."
88308,521570,4.0,4.0,76561198329548331,"{'item_id': '521570', 'item_name': 'You Have 1..."
88308,519140,3.0,3.0,76561198329548331,"{'item_id': '519140', 'item_name': 'Minds Eyes..."


In [194]:
#Se eliminan los datos nulos por columnas
data_it3 = data_it3.dropna(subset=['item_id'])
data_it3 = data_it3.dropna(subset=['playtime_forever'])

In [195]:
#Se eliminan 2 columnas adicionales
data_it3.drop(['items','playtime_2weeks'], axis=1, inplace=True)

In [196]:
data_it3

,item_id,playtime_forever,user_id
0,10,6.0,76561197970982479
0,20,0.0,76561197970982479
0,30,7.0,76561197970982479
0,40,0.0,76561197970982479
0,50,0.0,76561197970982479
...,...,...,...
88308,346330,0.0,76561198329548331
88308,373330,0.0,76561198329548331
88308,388490,3.0,76561198329548331
88308,521570,4.0,76561198329548331


Se define el dataframe solo para aquellos juegos donde tuvieron un tiempo jugado (condicion)

In [197]:
data_it3 = data_it3[data_it3['playtime_forever'] != 0]

Se conservan las 3 columnas para el analisis

In [198]:
data_it3 = data_it3[['user_id','item_id','playtime_forever']]
data_it3

,user_id,item_id,playtime_forever
0,76561197970982479,10,6.0
0,76561197970982479,30,7.0
0,76561197970982479,300,4733.0
0,76561197970982479,240,1853.0
0,76561197970982479,3830,333.0
...,...,...,...
88308,76561198329548331,304930,677.0
88308,76561198329548331,227940,43.0
88308,76561198329548331,388490,3.0
88308,76561198329548331,521570,4.0


Se guarda como formato Parquet y CSV para el merge

In [199]:
data_it3.to_parquet('user_items.parquet', index= False)

In [200]:
data_it3.to_csv('user_items.csv', index=False)

### ETL de `australian_user_reviews`

In [201]:
data_list1 = []
#Ruta del archivo JSON
file_path = 'australian_user_reviews.json'

#Abrir el archivo y procesar cada línea
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        try:
            # Usar ast.literal_eval para convertir la línea en un diccionario
            json_data = ast.literal_eval(line)
            data_list1.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

#Crear un DataFrame a partir de la lista de diccionarios
data_it = pd.DataFrame(data_list1)

In [202]:
data_it

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."
...,...,...,...
25794,76561198306599751,http://steamcommunity.com/profiles/76561198306...,"[{'funny': '', 'posted': 'Posted May 31.', 'la..."
25795,Ghoustik,http://steamcommunity.com/id/Ghoustik,"[{'funny': '', 'posted': 'Posted June 17.', 'l..."
25796,76561198310819422,http://steamcommunity.com/profiles/76561198310...,"[{'funny': '1 person found this review funny',..."
25797,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"[{'funny': '', 'posted': 'Posted July 21.', 'l..."


In [203]:
data_it1 = data_it.explode(['reviews'])
data_it2 = pd.json_normalize(data_it1['reviews']).set_index(data_it1['reviews'].index)
data_it3= pd.concat([data_it2, data_it1], axis=1)

In [204]:
data_it3

,funny,posted,last_edited,item_id,helpful,recommend,review,user_id,user_url,reviews
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,js41637,http://steamcommunity.com/id/js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...,...,...,...,...,...,...
25797,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,76561198312638244,http://steamcommunity.com/profiles/76561198312...,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,,Posted July 20.,,730,No ratings yet,True,:D,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [205]:
#Se eliminan las columnas que no se consideran relevantes para el posterior analisis
data_it3.drop(['funny','last_edited','helpful','user_url'], axis=1, inplace=True)

In [206]:
data_it3

,posted,item_id,recommend,review,user_id,reviews
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...,...,...
25797,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,Posted July 20.,730,True,:D,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [207]:
#Se eliminan los datos nulos por columnas
#se eliminan los datos nulos por columnas
data_it3 = data_it3.dropna(subset=['posted'])
data_it3 = data_it3.dropna(subset=['item_id'])

In [208]:
data_it3

,posted,item_id,recommend,review,user_id,reviews
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479,"{'funny': '', 'posted': 'Posted November 5, 20..."
0,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479,"{'funny': '', 'posted': 'Posted July 15, 2011...."
0,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,"{'funny': '', 'posted': 'Posted April 21, 2011..."
1,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637,"{'funny': '', 'posted': 'Posted June 24, 2014...."
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637,"{'funny': '', 'posted': 'Posted September 8, 2..."
...,...,...,...,...,...,...
25797,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244,"{'funny': '', 'posted': 'Posted July 10.', 'la..."
25797,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244,"{'funny': '', 'posted': 'Posted July 8.', 'las..."
25798,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,"{'funny': '1 person found this review funny', ..."
25798,Posted July 20.,730,True,:D,LydiaMorley,"{'funny': '', 'posted': 'Posted July 20.', 'la..."


In [209]:
#Se eliminan 2 columnas adicionales
data_it3.drop(['reviews'], axis=1, inplace=True)

In [210]:
#se define el dataframe solo para aquellos juegos donde tuvieron un tiempo jugado (condicion)
data_it3 = data_it3[data_it3['item_id'] != 0]


In [211]:
data_it3


,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
0,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
0,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
1,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637
...,...,...,...,...,...
25797,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244
25797,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244
25798,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley
25798,Posted July 20.,730,True,:D,LydiaMorley


In [212]:
#Se eliminan los duplicados y valores nulos en caso de haber
data_it3 = data_it3.drop_duplicates()
data_it3 = data_it3.dropna()

In [213]:
#Parseamos la columna de la fecha #Posted#
def convertir_a_fecha(fecha_str):
    try:
        fecha_str = fecha_str.replace('Posted ', '')

        # Intentar analizar la fecha
        fecha = parser.parse(fecha_str, fuzzy=True)
        
        # Si no se especifica el año, establecerlo como "ND"
        if fecha.year == parser.DEFAULTPARSER:
            return "ND"
        
        return fecha
    except:
        return None

In [214]:
data_it3['posted'] = data_it3['posted'].apply(convertir_a_fecha)

In [215]:
data_it3

,posted,item_id,recommend,review,user_id
0,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
0,2011-07-15,22200,True,It's unique and worth a playthrough.,76561197970982479
0,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
1,2014-06-24,251610,True,I know what you think when you see this title ...,js41637
1,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,js41637
...,...,...,...,...,...
25797,2024-07-10,70,True,a must have classic from steam definitely wort...,76561198312638244
25797,2024-07-08,362890,True,this game is a perfect remake of the original ...,76561198312638244
25798,2024-07-03,273110,True,had so much fun plaing this and collecting res...,LydiaMorley
25798,2024-07-20,730,True,:D,LydiaMorley


Se guarda como formato .Parquet para el merge

In [216]:
data_it3.to_parquet('user_reviews.parquet', index=False)

Se guarda como formato .CSV para el merge

In [217]:
data_it3.to_csv('user_reviews.csv', index=False)

### Analisis de sentimiento sobre la columna Review 

In [3]:
# Instalamos e importamos
!pip install textblob
from textblob import TextBlob

Defaulting to user installation because normal site-packages is not writeable


In [219]:
def analyze_sentiment(review):
    analysis = TextBlob(str(review))
    # Se Asigna un valor según el análisis de sentimiento
    if analysis.sentiment.polarity > 0.1: 
        return 2  # Positivo
    elif analysis.sentiment.polarity < -0.1:
        return 0  # Malo
    else:
        return 1  # Neutral

Se agrega la columna Sentiment_analysis

In [220]:
data_it3['Sentiment_analysis'] = data_it3['review'].apply(analyze_sentiment)

In [221]:
data_it3

,posted,item_id,recommend,review,user_id,Sentiment_analysis
0,2011-11-05,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2
0,2011-07-15,22200,True,It's unique and worth a playthrough.,76561197970982479,2
0,2011-04-21,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479,1
1,2014-06-24,251610,True,I know what you think when you see this title ...,js41637,2
1,2013-09-08,227300,True,For a simple (it's actually not all that simpl...,js41637,1
...,...,...,...,...,...,...
25797,2024-07-10,70,True,a must have classic from steam definitely wort...,76561198312638244,2
25797,2024-07-08,362890,True,this game is a perfect remake of the original ...,76561198312638244,2
25798,2024-07-03,273110,True,had so much fun plaing this and collecting res...,LydiaMorley,1
25798,2024-07-20,730,True,:D,LydiaMorley,2


Se guarda con el sentiment analisis

In [222]:
data_it3.to_parquet('user_reviews.parquet', index=False)

In [223]:
data_it3.to_csv('user_reviews.csv', index=False)

Volvemos a llamar a los CSV para la creacion de una tabla unica con un JOIN

In [224]:
dfgames = pd.read_csv('output_steam_games.csv')
dfitems = pd.read_csv('user_items.csv')
dfreviews = pd.read_csv('user_reviews.csv')

Doble Check si existen datos faltantes

In [225]:
dfgames = dfgames.dropna()
dfitems = dfitems.dropna()
dfreviews = dfreviews.dropna()

Unimos los datos dataset de User.

In [226]:
df_merge = pd.merge(dfitems,dfreviews, how="left")

In [227]:
#Doble Check si existen datos faltantes
df_merge = df_merge.dropna()
df_merge

,user_id,item_id,playtime_forever,posted,recommend,review,Sentiment_analysis
27,76561197970982479,22200,271.0,2011-07-15,True,It's unique and worth a playthrough.,2.0
29,76561197970982479,1250,10006.0,2011-11-05,True,Simple yet with great replayability. In my opi...,2.0
59,76561197970982479,43110,834.0,2011-04-21,True,Great atmosphere. The gunplay can be a bit chu...,1.0
331,js41637,227300,551.0,2013-09-08,True,For a simple (it's actually not all that simpl...,1.0
387,js41637,239030,349.0,2013-11-29,True,Very fun little game to play when your bored o...,1.0
...,...,...,...,...,...,...,...
3285183,Ghoustik,730,3969.0,2024-06-17,True,Gra naprawdę fajna.Ale jest kilka rzeczy do kt...,1.0
3285197,76561198312638244,70,1010.0,2024-07-10,True,a must have classic from steam definitely wort...,2.0
3285198,76561198312638244,130,221.0,2024-07-10,True,if you liked Half life i would really recommen...,2.0
3285210,76561198312638244,233270,533.0,2024-07-21,True,this is a very fun and nice 80s themed shooter...,2.0


Renombro la columna id de output por item_id para que se pueda relacionar

In [228]:
dfgames.rename(columns={'id': 'item_id'}, inplace=True)

In [229]:
dfgames

,publisher,genres,app_name,release_date,price,item_id,developer
0,2K Games,['Strategy'],Railroad Tycoon II Platinum,1998-10-31,4.99,7620.0,PopTop
1,2K Games,['Strategy'],X-COM: Terror From the Deep,1995-04-01,4.99,7650.0,"MicroProse Software, Inc"
2,"Square Enix, Feral Interactive (Mac)","['Action', 'Adventure']",Tomb Raider: Anniversary,2007-06-05,8.99,8000.0,"Crystal Dynamics,Feral Interactive (Mac)"
3,Capcom,['Action'],Devil May Cry® 3 Special Edition,2006-05-23,19.99,6550.0,"CAPCOM Co., Ltd."
4,Telltale Games,"['Action', 'Adventure']",Sam & Max 101: Culture Shock,2006-10-17,19.99,8200.0,Telltale Games
...,...,...,...,...,...,...,...
22762,Bidoniera Games,"['Action', 'Adventure', 'Casual', 'Indie']",Kebab it Up!,2018-01-04,1.99,745400.0,Bidoniera Games
22763,Ghost_RUS Games,"['Casual', 'Indie', 'Simulation', 'Strategy']",Colony On Mars,2018-01-04,1.99,773640.0,"Nikita ""Ghost_RUS"""
22764,Sacada,"['Casual', 'Indie', 'Strategy']",LOGistICAL: South Africa,2018-01-04,4.99,733530.0,Sacada
22765,Laush Studio,"['Indie', 'Racing', 'Simulation']",Russian Roads,2018-01-04,1.99,610660.0,Laush Dmitriy Sergeevich


In [230]:
df_merge = pd.merge(dfgames,df_merge, how="left")

In [231]:
df_merge = df_merge.dropna()

In [232]:
df_merge

,publisher,genres,app_name,release_date,price,item_id,developer,user_id,playtime_forever,posted,recommend,review,Sentiment_analysis
2,"Square Enix, Feral Interactive (Mac)","['Action', 'Adventure']",Tomb Raider: Anniversary,2007-06-05,8.99,8000.0,"Crystal Dynamics,Feral Interactive (Mac)",76561198052998873,378.0,2015-12-11,True,"One of the best games of all time, I have been...",2.0
3,Capcom,['Action'],Devil May Cry® 3 Special Edition,2006-05-23,19.99,6550.0,"CAPCOM Co., Ltd.",ryanflyin13,2872.0,2024-06-04,True,This game made a huge impact on my life when i...,1.0
4,Capcom,['Action'],Devil May Cry® 3 Special Edition,2006-05-23,19.99,6550.0,"CAPCOM Co., Ltd.",SambaWarKiddo,47.0,2014-05-31,True,"Fantastic game by absolutely all means, and if...",2.0
5,Capcom,['Action'],Devil May Cry® 3 Special Edition,2006-05-23,19.99,6550.0,"CAPCOM Co., Ltd.",76561198104824869,4169.0,2014-02-22,True,虽然不支持控制器和不能全银幕的游戏，但并不会减少游戏的可玩性和热趣，你依然可以灵活的进行华丽...,1.0
6,Capcom,['Action'],Devil May Cry® 3 Special Edition,2006-05-23,19.99,6550.0,"CAPCOM Co., Ltd.",76561198096849086,776.0,2014-07-01,True,.Una compra recomendada para los que les gusta...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56781,Valve,['Action'],Counter-Strike: Condition Zero,2004-03-01,9.99,80.0,Valve,76561198015050660,2416.0,2014-12-23,False,作為第一個在steam入手的遊戲，老實說當初是為了1.6來順便帶回來，由於當時網路不太穩的關...,1.0
56782,Valve,['Action'],Counter-Strike: Condition Zero,2004-03-01,9.99,80.0,Valve,76561198023508728,15.0,2014-10-26,False,i caused 9/11,1.0
56783,Valve,['Action'],Counter-Strike: Condition Zero,2004-03-01,9.99,80.0,Valve,green290,442.0,2015-01-05,True,This game was suprisingly my first FPS game.11...,2.0
56784,Valve,['Action'],Counter-Strike: Condition Zero,2004-03-01,9.99,80.0,Valve,174gamecuman700kngkakak,72.0,2014-10-31,True,:v,1.0


Se crea el CSV y se ubica en la carpeta data asi los proximos CSV se alojen en el mismo lugar

In [233]:
df_merge.to_parquet('df_merge.parquet', index=False)

In [234]:
df_merge = pd.read_parquet('df_merge.parquet')

In [235]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36579 entries, 0 to 36578
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   publisher           36579 non-null  object 
 1   genres              36579 non-null  object 
 2   app_name            36579 non-null  object 
 3   release_date        36579 non-null  object 
 4   price               36579 non-null  float64
 5   item_id             36579 non-null  float64
 6   developer           36579 non-null  object 
 7   user_id             36579 non-null  object 
 8   playtime_forever    36579 non-null  float64
 9   posted              36579 non-null  object 
 10  recommend           36579 non-null  bool   
 11  review              36579 non-null  object 
 12  Sentiment_analysis  36579 non-null  float64
dtypes: bool(1), float64(4), object(8)
memory usage: 3.4+ MB


Se cambia el tipo de dato de Free to play a 0 para poder hacer el analisis

In [236]:
df_merge['price'] = df_merge['price'].replace('Free to Play', 0)

Convertir la columna 'price' a tipo numérico y reemplazar NaN con ceros

In [237]:
# Convertir la columna 'price' a tipo numérico y reemplazar NaN con ceros
df_merge['price'] = pd.to_numeric(df_merge['price'], errors='coerce').fillna(0)

# Imprimir los valores no numéricos en la columna 'price' después de la conversión
print("Valores no numéricos en la columna 'price' después de la conversión:")
print(df_merge.loc[df_merge['price'].isnull(),'price'])

Valores no numéricos en la columna 'price' después de la conversión:
Series([], Name: price, dtype: float64)


Se configura la columna release_date a un valor de años

In [238]:
# Supongamos que 'release_date' es la columna que contiene las fechas
df_merge['release_date'] = pd.to_datetime(df_merge['release_date'], errors='coerce')

# Asegúrate de que no haya valores nulos en la columna 'release_date'
print(df_merge['release_date'].isnull().sum())

# Crea la columna 'Años' extrayendo el año después de la conversión
df_merge['Años'] = df_merge['release_date'].dt.year

# Imprime el DataFrame para verificar los resultados
print(df_merge)

0
                                  publisher                   genres  \
0      Square Enix, Feral Interactive (Mac)  ['Action', 'Adventure']   
1                                    Capcom               ['Action']   
2                                    Capcom               ['Action']   
3                                    Capcom               ['Action']   
4                                    Capcom               ['Action']   
...                                     ...                      ...   
36574                                 Valve               ['Action']   
36575                                 Valve               ['Action']   
36576                                 Valve               ['Action']   
36577                                 Valve               ['Action']   
36578                                 Valve               ['Action']   

                               app_name release_date  price  item_id  \
0              Tomb Raider: Anniversary   2007-06-05   8.99  

Configuramos el modelo con 4000 registros para poder manipularlo por cuestiones de peso en render y github

In [239]:
modelo = df_merge.head(4000)

Creamos el Dataset del modelo de entrenamiento

In [240]:
modelo.to_parquet('modelo.parquet', index=False)

Se guarda el Parquet final luego de todo el Preprocesamiento de los datos

In [241]:
df_merge.to_parquet('df_merge.parquet', index=False)

### Hacemos el entrenamiento del modelo de recomendacion

In [242]:
def entrenar_modelo():
    # Cargar los datos desde los CSVs
    user_items = pd.read_csv('user_items.csv')
    user_reviews = pd.read_csv('user_reviews.csv')
    output = pd.read_csv('output_steam_games.csv')
    
    # Unir las tablas user_items y user_reviews basadas en la columna 'user_id'
    merged_data = pd.merge(user_items, user_reviews, how="inner")
    #df= merged_data.drop_duplicates()
    df = merged_data.sample(n=5000, random_state=42).drop_duplicates()
   
    promedio_playtime = df['playtime_forever'].mean()
   
    distancia_chica = promedio_playtime / 3
    distancia_media = promedio_playtime - (distancia_chica * 2)

    def asignar_valor(playtime):
        if playtime <= distancia_chica:
            return 0
        elif distancia_chica < playtime <= distancia_media:
            return 1
        else:
            return 2

    # Aplica la función a la columna 'playtime forever' para crear la nueva columna 'nueva_columna'
    df['playtime'] = df['playtime_forever'].apply(asignar_valor)

    user_id_mapping = {user_id: i for i, user_id in enumerate(df['user_id'].unique())}
    df['user_id_numeric'] = df['user_id'].map(user_id_mapping)

    # Mapear item_id a identificadores numéricos únicos
    item_id_mapping = {item_id: i for i, item_id in enumerate(df['item_id'].unique())}
    df['item_id_numeric'] = df['item_id'].map(item_id_mapping)

    # Suma la columna 'playtime_forever' al 'Sentiment_analysis'
    df['combined_rating'] = df['Sentiment_analysis'] + df['playtime']
    
    # Utiliza pivot para crear una matriz de recomendación
    matriz_recomendacion = df.pivot(index='user_id_numeric', columns='item_id_numeric', values='combined_rating')

    # Llenar los valores NaN con 0
    matriz_recomendacion = matriz_recomendacion.fillna(0)

    # Se obtienen los valores de la matriz como un array
    ratings = matriz_recomendacion.values

    # División en conjuntos de entrenamiento y prueba
    ratings_train, _ = train_test_split(ratings, test_size=0.2, random_state=42)

    sim_matrix = 1 - cosine_distances(ratings_train)

    # Número de usuarios y elementos
    num_users, _ = ratings_train.shape

    # Selecciona una parte de la matriz de similitud para el conjunto de entrenamiento
    sim_matrix_train = sim_matrix[:num_users, :num_users]

    datos_entrenamiento = (sim_matrix_train, ratings_train, user_id_mapping, item_id_mapping, df, output)
    
    # Guarda los datos de entrenamiento en un archivo usando pickle
    with gzip.open('datos_entrenamiento.pkl.gz', 'wb') as f:
        pickle.dump(datos_entrenamiento, f)
    
    model = sim_matrix_train, ratings_train, user_id_mapping, item_id_mapping, df, output
    
    return model
entrenar_modelo()

(array([[1., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 1.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 {'obscenedagger': 0,
  'rrbr': 1,
  '76561198069547848': 2,
  '76561198147285284': 3,
  'howsitgoinaz': 4,
  'halofan360': 5,
  'mailiam123': 6,
  'neochuah94': 7,
  'fui312': 8,
  'DABLUICE': 9,
  'ScoutCounterAttack': 10,
  'bloodyjak': 11,
  'darrylcheng': 12,
  'horizon1101': 13,
  '76561198072672804': 14,
  'Playerupjack': 15,
  'pizzerman': 16,
  'NoSwagm8': 17,
  'IISantaII': 18,
  '76561198050243948': 19,
  '76561198083176167': 20,
  '76561198028640914': 21,
  '76561198084634633': 22,
  '